# Door Dash Data Exploratory Data Analysis


This notebook will 'explore' the dataset we are working with before making any assumptions. This will help guide my decisions when 'transforming' my raw formatted data into production level data so it can be stored into our data warehouse (SnowFlake)

In [1]:
## Import common packages and 
import pandas as pd
import numpy as np
import boto3
from utils import io
import config





,Unnamed: 0,Customer_placed_order_datetime,Placed_order_with_restaurant_datetime,Driver_at_restaurant_datetime,Delivered_to_consumer_datetime,Driver_ID,Restaurant_ID,Consumer_ID,Is_New,Delivery_Region,Is_ASAP,Order_total,Amount_of_discount,Amount_of_tip,Refunded_amount,Total_time_elapsed,Day
0,0,14 20:27:45,14 20:29:41,14 20:39:32,14 20:52:03,86,12,5,False,Palo Alto,True,20.45,20.45,3.07,0.00,0 days 00:24:18.000000000,14
1,1,07 20:16:28,07 20:17:32,07 20:36:00,07 20:49:02,325,66,5,False,Palo Alto,True,40.62,40.62,3.73,0.00,0 days 00:32:34.000000000,7
2,2,13 19:35:09,13 19:39:26,13 20:28:16,13 20:52:44,200,124,5,False,Palo Alto,True,37.78,37.78,1.89,0.00,0 days 01:17:35.000000000,13
3,3,22 19:47:53,22 19:56:08,22 20:01:20,22 20:18:01,154,5,14,False,Palo Alto,True,39.66,0.00,1.98,0.00,0 days 00:30:08.000000000,22
4,4,03 19:01:52,03 19:09:08,03 19:36:20,03 19:45:26,332,9,14,False,Palo Alto,True,39.66,0.00,5.95,0.00,0 days 00:43:34.000000000,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18073,18073,20 00:39:30,20 00:40:31,NaN,20 01:17:29,36,270,96518,False,San Jose,True,44.59,0.00,3.54,13.58,0 days 00:37:59.000000000,20
18074,18074,26 19:59:56,26 20:05:45,NaN,26 21:00:14,264,294,96518,False,San Jose,True,19.03,0.00,1.19,20.22,0 days 01:00:18.000000000,26
18075,18075,30 18:20:01,30 18:24:13,30 18:57:09,30 19:31:20,232,294,96518,False,San Jose,True,23.93,0.00,0.82,24.75,0 days 01:11:19.000000000,30
18076,18076,27 21:48:21,27 21:57:50,NaN,27 22:36:59,300,370,100694,False,Mountain View,True,31.45,0.00,0.00,0.00,0 days 00:48:38.000000000,27


#

# Loading our Data

Lets take a look at the raw data. We can take a look at we we are dealing with by first listing out all raw data files in our bucket

In [2]:
print(io.listfiles('raw'))

['raw/1.csv', 'raw/10.csv', 'raw/11.csv', 'raw/12.csv', 'raw/13.csv', 'raw/14.csv', 'raw/15.csv', 'raw/16.csv', 'raw/17.csv', 'raw/18.csv', 'raw/19.csv', 'raw/2.csv', 'raw/20.csv', 'raw/21.csv', 'raw/22.csv', 'raw/23.csv', 'raw/24.csv', 'raw/25.csv', 'raw/26.csv', 'raw/27.csv', 'raw/28.csv', 'raw/29.csv', 'raw/3.csv', 'raw/30.csv', 'raw/31.csv', 'raw/4.csv', 'raw/5.csv', 'raw/6.csv', 'raw/7.csv', 'raw/8.csv', 'raw/9.csv']


Lets look at the schema of these tables. We can import a csv file using the imp_file from the io module. We simply give it the directory of the file and the file name itself.

In [3]:
## Loading in dataframe of day 2
df = io.imp_file('raw','2')
df



,Unnamed: 0.1,Unnamed: 0,Customer_placed_order_datetime,Placed_order_with_restaurant_datetime,Driver_at_restaurant_datetime,Delivered_to_consumer_datetime,Driver_ID,Restaurant_ID,Consumer_ID,Is_New,Delivery_Region,Is_ASAP,Order_total,Amount_of_discount,Amount_of_tip,Refunded_amount,Total_time_elapsed,Day
0,7,7,02 02:13:30,02 02:21:03,02 02:28:26,02 03:17:48,29,53,14,False,Palo Alto,True,52.71,0.0,4.29,0.0,0 days 01:04:18.000000000,2
1,10,10,02 17:14:25,02 17:20:33,02 18:35:25,02 18:47:56,69,312,14,False,Palo Alto,False,31.89,0.0,3.56,0.0,0 days 01:33:31.000000000,2
2,40,40,02 01:45:10,02 01:45:35,02 02:01:11,02 02:17:59,133,10,35,False,Palo Alto,True,73.32,0.0,3.09,0.0,0 days 00:32:49.000000000,2
3,76,76,02 04:20:09,02 04:21:17,02 04:34:49,02 05:01:03,343,5,105,False,Palo Alto,True,45.10,0.0,2.26,0.0,0 days 00:40:54.000000000,2
4,124,124,02 01:25:41,02 01:28:36,NaN,02 02:13:16,259,63,178,False,Palo Alto,True,62.98,0.0,3.15,0.0,0 days 00:47:35.000000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
569,14078,14078,02 18:51:04,02 19:23:36,02 19:32:22,02 20:05:44,262,69,77942,False,Palo Alto,False,52.00,6.0,2.11,0.0,0 days 01:14:40.000000000,2
570,14080,14080,02 18:43:19,02 18:46:54,02 19:46:19,02 20:09:48,41,107,77988,False,Palo Alto,False,151.74,6.0,12.85,0.0,0 days 01:26:29.000000000,2
571,14089,14089,02 19:26:04,02 19:28:50,02 19:37:00,02 19:55:23,323,30,78089,False,Mountain View,True,22.31,6.0,2.00,0.0,0 days 00:29:19.000000000,2
572,14098,14098,02 20:42:38,02 20:43:20,02 20:54:18,02 21:04:49,334,26,78194,False,Mountain View,True,14.65,6.0,1.19,0.0,0 days 00:22:11.000000000,2


In [4]:
#Schema
df.dtypes


Unnamed: 0.1                               int64
Unnamed: 0                                 int64
Customer_placed_order_datetime            object
Placed_order_with_restaurant_datetime     object
Driver_at_restaurant_datetime             object
Delivered_to_consumer_datetime            object
Driver_ID                                  int64
Restaurant_ID                              int64
Consumer_ID                                int64
Is_New                                      bool
Delivery_Region                           object
Is_ASAP                                     bool
Order_total                              float64
Amount_of_discount                       float64
Amount_of_tip                            float64
Refunded_amount                          float64
Total_time_elapsed                        object
Day                                        int64
dtype: object

# Exploratory Data Analysis

